In [ ]:
#Importing required libraries
import cv2
import torch
import numpy as np
import torch.nn as nn
import tensorflow as tf
import torch.optim as optim
from torchvision import models
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from keras.datasets import fashion_mnist

#Initializing device details and Importing the dataset
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()
(xTrain, yTrain), (xTest, yTest) = fashion_mnist.load_data()

In [ ]:
plt.imshow(xTrain[0]/27, cmap = 'gray')

In [ ]:
#Dataset Preprocessing
length, breadth = xTrain[0].shape
flattenDim = length * breadth
xTrain = torch.FloatTensor(np.round(xTrain / 27).astype(int)).to(device)
xTest = np.round(xTest / 27).astype(int)
#xTrainFlat = torch.FloatTensor(np.reshape(xTrain, (-1,flattenDim))).to(device)
xTestFlat = np.reshape(xTest, (-1,flattenDim)) 

In [ ]:
#Dataloader
trainLoader = DataLoader(dataset = xTrain, batch_size = 1, shuffle = True)

In [ ]:
#Autoencoder NN Module consisting of Encoder and Decoder classes
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.Encoder = nn.Sequential(
        nn.Conv2d(in_channels = 1, out_channels = 16, kernel_size = 3),
        nn.ReLU(),
        nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size = 3),
        nn.ReLU(),
        nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 5),
        nn.ReLU(),
        nn.Linear(20,9),
        nn.ReLU()
        )
        self.Decoder = nn.Sequential(
        nn.Linear(9,20),
        nn.ReLU(),
        nn.ConvTranspose2d(in_channels = 64, out_channels = 32, kernel_size = 5),
        nn.ReLU(),
        nn.ConvTranspose2d(in_channels = 32, out_channels = 16, kernel_size = 3),
        nn.ReLU(),
        nn.ConvTranspose2d(in_channels = 16, out_channels = 1, kernel_size = 3),
        nn.ReLU(),
        nn.Softmax(dim = 1)    
        )
        
    def forward(self, x):
        encoderOutput = self.Encoder(x)
        decoderOutput = self.Decoder(encoderOutput)
        return encoderOutput, decoderOutput

In [ ]:
model = autoencoder().to(device)
modelParameters = list(model.parameters())
criterion = nn.MSELoss(reduction = 'mean')
optimizer = optim.SGD(modelParameters, lr = (1e-2), momentum=0.5)

encoderOutputs = []
decoderOutputs = []
for epoch in range(100):
  losses = []
  for batchIndex, batchImage in enumerate(trainLoader):
    batchImage = batchImage.to(device)
    batchImage = batchImage.unsqueeze(1)
    encoderOutput, decoderOutput = model(batchImage)
    loss = criterion(decoderOutput, batchImage)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    losses.append(loss.cpu().detach().numpy())
    encoderOutputs.append((epoch, batchIndex, encoderOutput))
    decoderOutputs.append((epoch, batchIndex, decoderOutput))
  print("Epoch: ",epoch,"| Average loss: ",np.round(np.average(losses), 3),"| Lowest Loss: ",np.round(np.amin(losses), 3))
  torch.save(model.state_dict(), "AE.pth")
 
plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.plot(losses[-100:])

In [ ]:
encOutput, decOutput = model(torch.FloatTensor(xTest[100]).unsqueeze(0).to(device))

In [ ]:
plt.imshow(decOutput.cpu().detach().numpy().reshape(28, 28))

In [ ]:
print(len(decoderOutputs))